# Yolov4 Pytorch 1.7 for Edge Devices with Amazon SageMaker

With this notebook you'll train a **Yolov4** model with Amazon SageMaker and then, optimize it for Jetson Xavier.


In [ ]:
import sagemaker
import numpy as np
import glob
import os
from sagemaker.pytorch.estimator import PyTorch

role = sagemaker.get_execution_role()
sagemaker_session=sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()
img_size=416
model_type='' # tiny
model_name='yolov4'if model_type=='' else f"yolov4-{model_type}"

### Download a public implementation of Yolov4 for Pytorch (Author: Wong Kin Yiu)

In [ ]:
if not os.path.isdir('yolov4'):
    !git clone https://github.com/WongKinYiu/PyTorch_YOLOv4 yolov4
    !cd yolov4 && git checkout 3c42cbd1b0fa28ad19436d01e0e240404463ff80 && git apply ../mish.patch
    !echo 'tensorboard' > yolov4/requirements.txt

### Prepare a Python script that will be the entrypoint of the training process
This script will wrap the original training scripts and expose the parameters to SageMaker Estimator

In [ ]:
%%writefile yolov4/sagemaker_train.py
import sys
import subprocess
## We need to remove smdebug to avoid the Hook bug https://github.com/awslabs/sagemaker-debugger/issues/401
subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "smdebug"])
import os
import yaml
import argparse
import torch
import shutil
import urllib
from models.models import Darknet

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    
    parser.add_argument('--num-classes', type=int, default=80, help='Number of classes')
    parser.add_argument('--img-size', type=int, default=640, help='Size of the image')
    parser.add_argument('--epochs', type=int, default=1, help='Number of epochs')
    parser.add_argument('--batch-size', type=int, default=16, help='Batch size')
    parser.add_argument('--adam', action='store_true', help='use torch.optim.Adam() optimizer')
    parser.add_argument('--pretrained', action='store_true', help='use pretrained model')
    
    parser.add_argument('--model-dir', type=str, default=os.environ["SM_MODEL_DIR"], help='Trained model dir')
    parser.add_argument('--train', type=str, default=os.environ["SM_CHANNEL_TRAIN"], help='Train path')
    parser.add_argument('--train-suffix', type=str, default='', help='Train path suffix')
    parser.add_argument('--validation', type=str, default=os.environ["SM_CHANNEL_VALIDATION"], help='Validation path')
    parser.add_argument('--validation-suffix', type=str, default='', help='Validation path suffix')
    
    parser.add_argument('--model-type', type=str, choices=['', 'tiny'], default="", help='Model type')
    
    # hyperparameters
    with open('data/hyp.scratch.yaml', 'r') as f:
        hyperparams = yaml.load(f, Loader=yaml.FullLoader)    
    for k,v in hyperparams.items():
        parser.add_argument(f"--{k.replace('_', '-')}", type=float, default=v)
    
    args,unknown = parser.parse_known_args()
    
    base_path=os.path.dirname(__file__)
    project_dir = os.environ["SM_OUTPUT_DATA_DIR"]

    # prepare the hyperparameters metadat
    with open(os.path.join(base_path,'data', 'hyp.custom.yaml'), 'w' ) as y:
        y.write(yaml.dump({h:vars(args)[h] for h in hyperparams.keys()}))

    # prepare the training data metadata
    with open(os.path.join(base_path,'data', 'custom.yaml'), 'w') as y:
        y.write(yaml.dump({            
            'names': [f'class_{i}' for i in range(args.num_classes)],
            'train': os.path.join(args.train, args.train_suffix),
            'val': os.path.join(args.validation, args.validation_suffix),
            'nc': args.num_classes
        }))
    model_name = "yolov4" if len(args.model_type) == 0 else f"yolov4-{args.model_type}"
    # run the training script
    weights_file=''
    if args.pretrained:
        weights_file = f'weights/{model_name}.weights'
        urllib.request.urlretrieve(
            f'https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/{model_name}.weights',
            weights_file
        )

    train_cmd = [
        sys.executable, os.path.join(base_path,'train.py'),
        "--data", "custom.yaml",
        "--hyp", "hyp.custom.yaml",
        "--cfg", f"cfg/{model_name}.cfg",
        "--img", str(args.img_size),
        "--batch", str(args.batch_size),
        "--epochs", str(args.epochs),
        "--logdir", project_dir,
        "--weights", weights_file
    ]
    if args.adam: train_cmd.append("--adam")
    subprocess.check_call(train_cmd)
        
    # tracing and saving the model
    inp = torch.rand(1, 3, args.img_size, args.img_size).cpu()
    ckpt = torch.load(os.path.join(project_dir, 'exp0', 'weights', 'best.pt'), map_location='cpu')
    model = Darknet(f"cfg/{model_name}.cfg").cpu() # create
    model.load_state_dict(ckpt['model'], strict=False)
    p = model(inp)
    model_trace = torch.jit.trace(model, inp)
    model_trace.save(os.path.join(args.model_dir, 'model.pth'))

## Preparing the dataset
Here you'll download a sample dataset coco128. You can also replace this step with your own dataset. Just take a look on the labels format and create your dataset scructure following the same standard (COCO).

In [ ]:
import os
if not os.path.exists('coco128'):
    !wget -q https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip
    !unzip -q coco128.zip && rm -f coco128.zip
print('BBoxes annotation')
print('class x_center y_center width height')
!head coco128/labels/train2017/000000000009.txt

### Upload the dataset to S3

In [ ]:
prefix='data/coco128'
!rm -f coco128/labels/train2017.cache
train_path = sagemaker_session.upload_data('coco128', key_prefix=f'{prefix}/train')
val_path = sagemaker_session.upload_data('coco128', key_prefix=f'{prefix}/val')
print(train_path, val_path)

### Prepare the SageMaker Estimator to train the model

In [ ]:
estimator = PyTorch(
    'sagemaker_train.py',
    source_dir='yolov4',
    framework_version='1.7',
    role=role,
    sagemaker_session=sagemaker_session,
    instance_type='ml.p3.2xlarge',    
    instance_count=1,
    py_version='py3', 
    hyperparameters={
        'epochs': 20, # at least 2 epochs
        'batch-size': 8,
        'lr0': 0.0001,
        
        'pretrained': True, # transfer learning
        'num-classes': 80,
        'img-size': img_size,
        'model-type': model_type,
        
        # the final path needs to point to the images dir
        'train-suffix': 'images/train2017',
        'validation-suffix': 'images/train2017'
    }
)

In [ ]:
estimator.fit({'train': train_path, 'validation': val_path})

## Compile your trained model for the edge device
In this case, you will compile a model for Jetson Xavier Jetpack 4.4.1. Just change the parameters bellow to adjust the target to your own edge device.

## Finally create a SageMaker Edge Manager packaging job

In [ ]:
import time
import boto3
sm_client = boto3.client('sagemaker')
compilation_job_name = f'{model_name}-pytorch-{int(time.time()*1000)}'
sm_client.create_compilation_job(
    CompilationJobName=compilation_job_name,
    RoleArn=role,
    InputConfig={
        'S3Uri': f'{estimator.output_path}{estimator.latest_training_job.name}/output/model.tar.gz',
        'DataInputConfig': f'{{"input": [1,3,{img_size},{img_size}]}}',
        'Framework': 'PYTORCH'
    },
    OutputConfig={
        'S3OutputLocation': f's3://{sagemaker_session.default_bucket()}/{model_name}-pytorch/optimized/',
        'TargetPlatform': { 
            'Os': 'LINUX', 
            'Arch': 'ARM64', # change this to X86_64 if you need
            'Accelerator': 'NVIDIA'  # comment this if you don't have an Nvidia GPU
        },
        # Comment or change the following line depending on your edge device
        # Jetson Xavier: sm_72; Jetson Nano: sm_53
        'CompilerOptions': '{"trt-ver": "7.1.3", "cuda-ver": "10.2", "gpu-code": "sm_53"}' # Jetpack 4.4.1
    },
    StoppingCondition={ 'MaxRuntimeInSeconds': 900 }
)
while True:
    resp = sm_client.describe_compilation_job(CompilationJobName=compilation_job_name)    
    if resp['CompilationJobStatus'] in ['STARTING', 'INPROGRESS']:
        print('Running...')
    else:
        print(resp['CompilationJobStatus'], compilation_job_name)
        break
    time.sleep(5)


In [ ]:
import time
model_version = '1.0'
edge_packaging_job_name=f'{model_name}-pytorch-{int(time.time()*1000)}'
resp = sm_client.create_edge_packaging_job(
    EdgePackagingJobName=edge_packaging_job_name,
    CompilationJobName=compilation_job_name,
    ModelName=model_name,
    ModelVersion=model_version,
    RoleArn=role,
    OutputConfig={
        'S3OutputLocation': f's3://{bucket_name}/{model_name}'
    }
)
while True:
    resp = sm_client.describe_edge_packaging_job(EdgePackagingJobName=edge_packaging_job_name)    
    if resp['EdgePackagingJobStatus'] in ['STARTING', 'INPROGRESS']:
        print('Running...')
    else:
        print(resp['EdgePackagingJobStatus'], compilation_job_name)
        break
    time.sleep(5)